In [1]:
import torch
torch.__version__

'2.4.1+cpu'

## Simple NN Example

In [3]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs): 
        super().__init__()
        self.layers = torch.nn.Sequential(
        
        # 1st hidden layer
        torch.nn.Linear(num_inputs, 30), 
        torch.nn.ReLU(), 
        # 2nd hidden layer
        torch.nn.Linear(30, 20), 
        torch.nn.ReLU(),
        # output layer
        torch.nn.Linear(20, num_outputs),
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [4]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [5]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [8]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [12]:
print(model.layers[4].bias.shape)

torch.Size([3])


In [13]:
X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.0552,  0.3030,  0.0186]], grad_fn=<AddmmBackward0>)


In [14]:
Xt = torch.rand((1, 50))
with torch.no_grad():
    pred = torch.softmax(model(Xt), dim=1)
print(pred)

tensor([[0.2868, 0.4024, 0.3108]])


## Dataset Prep Example

In [15]:
X_train = torch.tensor([
 [-1.2, 3.1],
 [-0.9, 2.9],
 [-0.5, 2.6],
 [2.3, -1.1],
 [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])
X_test = torch.tensor([
 [-0.8, 2.8],
 [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [16]:
from torch.utils.data import Dataset
class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    def __getitem__(self, index): 
        one_x = self.features[index] 
        one_y = self.labels[index] 
        return one_x, one_y 
    def __len__(self):
        return self.labels.shape[0] 
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [17]:
from torch.utils.data import DataLoader
torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_ds, 
    batch_size=2,
    shuffle=True, 
    num_workers=0
    #drop_last = True
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False, 
    num_workers=0
)

## Training Loop

In [18]:
import torch.nn.functional as F

model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        logits = model(features)
        
        loss = F.cross_entropy(logits, labels) # Loss function
        
        optimizer.zero_grad() # Manually reset gradient to avoid accumulation
        loss.backward() # Calculate gradient with back prop
        optimizer.step() # Step in the gradient's direction
    
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval()
    # Optional model evaluation

Epoch: 001/003 | Batch 000/003 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/003 | Train/Val Loss: 0.65
Epoch: 001/003 | Batch 002/003 | Train/Val Loss: 0.42
Epoch: 002/003 | Batch 000/003 | Train/Val Loss: 0.05
Epoch: 002/003 | Batch 001/003 | Train/Val Loss: 0.13
Epoch: 002/003 | Batch 002/003 | Train/Val Loss: 0.00
Epoch: 003/003 | Batch 000/003 | Train/Val Loss: 0.01
Epoch: 003/003 | Batch 001/003 | Train/Val Loss: 0.00
Epoch: 003/003 | Batch 002/003 | Train/Val Loss: 0.02
